Reference:

1. https://machinelearningmastery.com/how-to-visualize-filters-and-feature-maps-in-convolutional-neural-networks/
2. https://cs231n.github.io/understanding-cnn/

# Draw ROC curve (AUC)

In [ ]:
from sklearn.metrics import roc_curve, auc

#use scores
y_score = model.predict(X_test)

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(nb_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

for i in range(nb_classes):
    plt.figure()
    plt.plot(fpr[i], tpr[i], color='orange', label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC - '+ category_names[i])
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
#all curves in oen chart
plt.figure(figsize=(10,10))

colors = ['aqua', 'darkorange', 'cornflowerblue', 'blue', 'orange', 'yellow', 'red', 'green', 'tan', 'black', 
         'dimgray', 'rosybrown', 'brown', 'silver', 'wheat', 'salmon', 'peru', 'lime', 'plum', 'indigo', 
         'gold']

for i, color in zip(range(number_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

# Draw Precision-Recall Curve

In [ ]:
precs = []
recalls = []
scores = model.predict_proba(X)
min_score = min([s[1] for s in scores])
max_score = max([s[1] for s in scores])
for threshold in np.arange(0, 1, 0.01):
    # prediction score above threshold is considered as positive
    if threshold < min_score or threshold > max_score:
        continue
    preds = [1 if s[1] >= threshold else 0 for s in scores]
    TP = sum([1 for gt, pred in zip(y, preds) if gt == 1 and pred == 1])
    FP = sum([1 for gt, pred in zip(y, preds) if gt == 0 and pred == 1])
    FN = sum([1 for gt, pred in zip(y, preds) if gt == 1 and pred == 0])
    if TP + FP == 0:
        precs.append(0.0)
    else:
        precs.append(TP / (TP + FP + 0.0))
    if TP + FN == 0:
        recalls.append(0.0)
    else:
        recalls.append(TP / (TP + FN + 0.0))

# plt.plot(recalls, precs, 'bs', ls='-')
plt.plot(recalls, precs, ls='-')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

# Draw comparison curves of training and validation sets in loss and accuracy

In [ ]:
def history_plot(model_id, history):
    plt.figure(figsize=(8,10)) 
    # plot loss
    plt.subplot(2, 1, 1)
    plt.title('Cross Entropy Loss - ' + model_id, fontsize = 12)
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='val')
    plt.xlabel("Epochs",fontsize = 12)
    plt.ylabel("Loss", fontsize = 12)
    plt.legend(loc='upper right')

    # plot accuracy
    plt.subplot(2, 1, 2)
    plt.title('Classification Accuracy ' + model_id, fontsize = 10)
    plt.plot(history.history['accuracy'], color='blue', label='train')
    plt.plot(history.history['val_accuracy'], color='orange', label='val')
    plt.xlabel("Epochs", fontsize = 120)
    plt.ylabel("Accuracy", fontsize = 12)
    plt.legend(loc='lower right')

    plt.show()

history_plot('CNN_base', history)

# Draw confusion matrix for multi-class problem

In [ ]:
def plot_cm(y_true, y_pred, figsize=(12,10)):
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%d/%d' % (c, s)
            elif c == 0:
                annot[i, j] = ''
            elif i<j:
                annot[i, j] = '%d' % (c)
            else:
                annot[i, j] = '%d' % (c)
    #cm = pd.DataFrame(cm, index=np.unique(y_true), columns=np.unique(y_true))
    cm = pd.DataFrame(cm, index=category_names, columns=category_names)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    sns.set(font_scale=1.4)
    sns.heatmap(cm, cmap= "YlGnBu", annot=annot, fmt='', ax=ax, annot_kws={"fontsize":12})

plot_cm( y_test.argmax(axis=1), predictions.argmax(axis=1)) #decode one hot encoding

# Draw filters (CNN)

The dark squares indicate small weights and the light squares represent large weights.

In [ ]:
# retrieve weights from the first conv layer
filters, biases = model.layers[0].get_weights()

# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

# plot first 6 filters
n_filters, ix = 6, 1

for i in range(n_filters):
    # get the filter
    print('Filter',i)
    plt.figure(figsize=(7,40))
    f = filters[:, :, :, i]
    # plot each channel separately
    for j in range(3):
        # specify subplot and turn of axis
        ax = plt.subplot(n_filters, 3, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        plt.imshow(f[:, :, j], cmap='gray')
        ix += 1
    # show the figure
    plt.show()

# Draw feature maps (CNN)

In [ ]:
#We only need output from the layers before the flatten
outputs = [model.layers[i].output for i in range(0,9)]
model_v = Model(inputs=model.inputs, outputs=outputs)
feature_maps = model_v.predict(image) #pass the image

for i in range(len(feature_maps)):
    print(i,feature_maps[i].shape)

# plot the output from each block
square = 8
i = 0
for fmap in feature_maps: 
    plt.figure(figsize=(20,max(20,20*fmap.shape[-1]//(square*square))))
    # plot all maps in an nx8 squares
    ix = 1
    print('Layer {}:{}'.format(i,model.layers[i].name))
    i+=1
    for _ in range(fmap.shape[-1]//square):
        for _ in range(square):
            # specify subplot and turn of axis
            ax = plt.subplot(fmap.shape[-1]//square, square, ix)
            ax.set_xticks([])
            ax.set_yticks([])
            # plot filter channel in grayscale
            plt.imshow(fmap[0, :, :, ix-1], cmap='gray')
            ix += 1
    # show the figure
    plt.show()